In [7]:
import os
import langchain

from langchain_groq import ChatGroq


pinecone_key = os.getenv("PINECONE_API_KEY")
groq_key = os.getenv("GROQ_API_KEY")

chat = ChatGroq(
    api_key=groq_key,
    model_name="llama3-8b-8192" 
)


In [8]:
from langchain_community.embeddings import HuggingFaceEmbeddings

# Cargar el modelo de embeddings desde Hugging Face
embed_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Obtener la cantidad de dimensiones del embedding de la palabra 'hola'
print(len(embed_model.embed_query('hola')))



C:\Users\SHIFTA-0164\AppData\Local\Temp\ipykernel_39568\1894570505.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embed_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
c:\LLM\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


384


In [25]:
from langchain.document_loaders import PyPDFLoader

# Función para cargar un solo archivo PDF
def read_cv(cv_path):
    loader = PyPDFLoader(cv_path)
    documents = loader.load()
    return documents

# Ruta exacta del archivo de tu CV
cv_path = r'c:\Users\SHIFTA-0164\Downloads\CV_GUSTAVO JULIAN RIVAS.pdf'

# Cargar el CV
doc_cv = read_cv(cv_path)

In [26]:
# Función para leer todos los PDFs de un directorio (para tesis)
def read_doc(directory):
    file_loader = PyPDFDirectoryLoader(directory)
    documents = file_loader.load()
    return documents

# Función para leer un único PDF (CV)
def read_cv(file_path):
    loader = PyPDFLoader(file_path)
    documents = loader.load()
    return documents



# Leer CV (archivo único)
cv_path = r'c:\Users\SHIFTA-0164\Downloads\CV_GUSTAVO JULIAN RIVAS.pdf'
doc_cv = read_cv(cv_path)

# Combinar documentos
total =  doc_cv

In [27]:
from langchain.vectorstores import Chroma
import os

# Supongamos que ya cargaste tu documento (doc_cv o total)
# doc_cv = [...]  # Lista de documentos Langchain

# Creamos o conectamos a una base local de vectores
vectorstore = Chroma.from_documents(
    documents=doc_cv,
    embedding=embed_model,
    persist_directory="./chroma_db"
)

# Guardar los vectores localmente
vectorstore.persist()

print("Documentos embebidos y almacenados en Chroma ✅")


Documentos embebidos y almacenados en Chroma ✅


In [29]:
import os
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_groq import ChatGroq
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

# Embeddings
embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Conectar al vectorstore ya creado
persist_dir = "./chroma_db"
vectorstore = Chroma(persist_directory=persist_dir, embedding_function=embedding)

# Buscar contexto relevante
query = "¿Cuál es el primer nombre?"
docs = vectorstore.similarity_search(query, k=3)
contexto = "\n".join([doc.page_content for doc in docs])

# Prompt
prompt = PromptTemplate(
    input_variables=["context", "question"],
    template="""
Usá el siguiente contexto para responder la pregunta de forma clara y precisa:

Contexto:
{context}

Pregunta:
{question}

Respuesta:"""
)

chat = ChatGroq(
    api_key=os.getenv("GROQ_API_KEY"),
    model_name="llama3-8b-8192"
)

qa_chain = LLMChain(llm=chat, prompt=prompt)

respuesta = qa_chain.run({"context": contexto, "question": query})
print("Respuesta generada por Groq:")
print(respuesta)


Respuesta generada por Groq:
La respuesta es GUSTAVO.


In [30]:
query = "¿Donde estudio informatica?"
docs = vectorstore.similarity_search(query, k=3)
contexto = "\n".join([doc.page_content for doc in docs])

# Prompt
prompt = PromptTemplate(
    input_variables=["context", "question"],
    template="""
Usá el siguiente contexto para responder la pregunta de forma clara y precisa:

Contexto:
{context}

Pregunta:
{question}

Respuesta:"""
)

chat = ChatGroq(
    api_key=os.getenv("GROQ_API_KEY"),
    model_name="llama3-8b-8192"
)

qa_chain = LLMChain(llm=chat, prompt=prompt)

respuesta = qa_chain.run({"context": contexto, "question": query})
print("Respuesta generada por Groq:")
print(respuesta)


Respuesta generada por Groq:
La respuesta es: Escuela Técnica N° 37, Hogar Naval Stella Maris (2006-2011) y Universidad de la Matanza (2012-2014).
